In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report

In [ ]:
BATCH_SIZE = 8
NUM_EPOCHS = 10
IMAGE_SIZE = 128
TEST_SPLIT = 0.2
RANDOM_SEED = 42

In [22]:
data_dir = "../data/RIA_2_CLASSES_CROPPED/"

# Transforms
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes
num_classes = len(class_names)

# Train/test split
torch.manual_seed(RANDOM_SEED)
test_size = int(TEST_SPLIT * len(dataset))
train_size = len(dataset) - test_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [23]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * (IMAGE_SIZE // 8) * (IMAGE_SIZE // 8), 128), nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

In [24]:
# Model, loss, optimizer
model = SimpleCNN(num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
print("Training...")
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.4f}")

Training...
Epoch 1/10, Loss: 1.6309
Epoch 2/10, Loss: 0.7605
Epoch 3/10, Loss: 0.4902
Epoch 4/10, Loss: 0.3350
Epoch 5/10, Loss: 0.2172
Epoch 6/10, Loss: 0.1082
Epoch 7/10, Loss: 0.0343
Epoch 8/10, Loss: 0.0343
Epoch 9/10, Loss: 0.0242
Epoch 10/10, Loss: 0.0061


In [25]:
# Evaluation
print("Evaluating...")
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=class_names))

Evaluating...
                      precision    recall  f1-score   support

  angle-back-on-left       0.86      0.90      0.88        21
 angle-back-on-right       0.88      0.75      0.81        28
 angle-front-on-left       0.90      0.88      0.89        52
angle-front-on-right       0.89      0.86      0.88        37
                back       0.91      0.91      0.91        23
               front       0.83      0.96      0.89        26
     profile-on-left       0.88      0.74      0.80        19
    profile-on-right       0.86      1.00      0.92        24

            accuracy                           0.88       230
           macro avg       0.88      0.88      0.87       230
        weighted avg       0.88      0.88      0.88       230



In [35]:
torch.save(model.state_dict(), "side_detection_v0.pth")